### LibriVox
[LibriVox](https://librivox.org)
LibreVox Free domai audio book

包含了一大堆免费音频录音

### OpenSLR
[OpenSLR](http://www.openslr.org/resources.php) 中有对LibriVox做了一些整理，可以提供下载

---

# Speech Recognition Example from mxnet
[speech recognition example](https://github.com/apache/incubator-mxnet/tree/master/example/speech_recognition)

Input data are described in a JSON file Libri_sample.json as followed.

{"duration": 2.9450625, "text": "and sharing her house which was near by", "key": "./Libri_sample/3830-12531-0030.wav"}
{"duration": 3.94, "text": "we were able to impart the information that we wanted", "key": "./Libri_sample/3830-12529-0005.wav"}

So we need to find a way to get duration from .flac file

#### following code is saved in flac_cal_duration.py

import struct

def bytes_to_int(bytes: list) -> int:
        result = 0
        for byte in bytes:
            result = (result << 8) + byte
        return result


def get_flac_duration(filename: str) -> float:
    """
    Returns the duration of a FLAC file in seconds
    https://xiph.org/flac/format.html
    """
    with open(filename, 'rb') as f:
        if f.read(4) != b'fLaC':
            raise ValueError('File is not a flac file')
        header = f.read(4)
        while len(header):
            meta = struct.unpack('4B', header)  # 4 unsigned chars
            block_type = meta[0] & 0x7f  # 0111 1111
            size = bytes_to_int(header[1:4])

            if block_type == 0:  # Metadata Streaminfo
                streaminfo_header = f.read(size)
                unpacked = struct.unpack('2H3p3p8B16p', streaminfo_header)
                """
                https://xiph.org/flac/format.html#metadata_block_streaminfo
                16 (unsigned short)  | The minimum block size (in samples)
                                       used in the stream.
                16 (unsigned short)  | The maximum block size (in samples)
                                       used in the stream. (Minimum blocksize
                                       == maximum blocksize) implies a
                                       fixed-blocksize stream.
                24 (3 char[])        | The minimum frame size (in bytes) used
                                       in the stream. May be 0 to imply the
                                       value is not known.
                24 (3 char[])        | The maximum frame size (in bytes) used
                                       in the stream. May be 0 to imply the
                                       value is not known.
                20 (8 unsigned char) | Sample rate in Hz. Though 20 bits are
                                       available, the maximum sample rate is
                                       limited by the structure of frame
                                       headers to 655350Hz. Also, a value of 0
                                       is invalid.
                3  (^)               | (number of channels)-1. FLAC supports
                                       from 1 to 8 channels
                5  (^)               | (bits per sample)-1. FLAC supports from
                                       4 to 32 bits per sample. Currently the
                                       reference encoder and decoders only
                                       support up to 24 bits per sample.
                36 (^)               | Total samples in stream. 'Samples'
                                       means inter-channel sample, i.e. one
                                       second of 44.1Khz audio will have 44100
                                       samples regardless of the number of
                                       channels. A value of zero here means
                                       the number of total samples is unknown.
                128 (16 char[])      | MD5 signature of the unencoded audio
                                       data. This allows the decoder to
                                       determine if an error exists in the
                                       audio data even when the error does not
                                       result in an invalid bitstream.
                """

                samplerate = bytes_to_int(unpacked[4:7]) >> 4
                sample_bytes = [(unpacked[7] & 0x0F)] + list(unpacked[8:12])
                total_samples = bytes_to_int(sample_bytes)
                duration = float(total_samples) / samplerate

                return duration
            header = f.read(4)



In [ ]:
from flac_cal_duration import get_flac_duration,get_duration_corpus

In [ ]:
path=!pwd

path=path.get_spstr()+r'/LibriSpeech/dev-clean/84/121123'
print(path)

#### following function: get_duration_corpus is saved in flac_cal_duration.py

def get_duration_corpus(path):
    whole_list=listdir(path)
    #whole_list=!ls $path
    flac_list=[item for item in whole_list if item[-4:]=='flac']
    corpus_list=[item for item in whole_list if item[-4:]!='flac']
    print(flac_list)
    print(corpus_list)
    
    corpus_dict={}
    flac_dict={}
    key_dict={}
    
    for flac in flac_list:
        duration=get_flac_duration(path+'/'+flac)
        key=flac[:-5]
        flac_dict[key]=duration
        key_dict[key]=path+'/'+flac
        
    with open(path+'/'+corpus_list[0]) as f:
        for line in f:
            items=line.split(' ')
            key=items[0]
            value=' '.join(items[1:]).strip('\n')
            corpus_dict[key]=value    
    
    return flac_dict,corpus_dict,key_dict
    

#### now it is to figure out how to route into all sub-directory

In [ ]:
from os import listdir
import json

In [ ]:
flac_dict={}
corpus_dict={}
key_dict={}

In [ ]:
with open('LibriSpeech_directories.txt') as f:
    for line in f:
        if line[0] != '#':
            path_tmp=line.strip('\n')
            print("processing %s"%path_tmp)
            tmp_flac,tmp_corpus,tmp_key=get_duration_corpus(path_tmp)
            
            flac_dict.update(tmp_flac)
            corpus_dict.update(tmp_corpus)
            key_dict.update(tmp_key)

In [ ]:
with open('LibriSpeech.json', 'w') as f:
    for key in flac_dict.keys():
        data={'duration':flac_dict[key],'text':corpus_dict[key],'key':key_dict[key]}
        json.dump(data,f)